In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, knn_graph
from torch_geometric.data import Data, DataLoader
from torch_geometric.transforms import KNNGraph
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch_geometric.transforms import KNNGraph
from torch_geometric.nn import knn_graph
from torch_geometric.data import Batch
import numpy as np
from collections import Counter  # Import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt

class ModelNetDataset:
    def __init__(self, root, classes, transform=None):
        self.root = root.replace("\\", "/")
        self.classes = classes
        self.transform = transform

    def load_data(self):
        data_list = []
        for cls in self.classes:
            cls_path = os.path.join(self.root, cls).replace("\\", "/")
            for split in ['train', 'test']:
                split_path = os.path.join(cls_path, split).replace("\\", "/")
                print(f"Checking path: {split_path}")  # Debugging print
                if os.path.exists(split_path):
                    for file_name in os.listdir(split_path):
                        if file_name.endswith('.off'):
                            file_path = os.path.join(split_path, file_name).replace("\\", "/")
                            print(f"Processing file: {file_path}")  # Debugging print
                            vertices, faces = self.load_off(file_path)
                            data = self.process_data(vertices, cls)
                            if self.transform:
                                data.pos = data.x  # Ensure data has 'pos' attribute
                                data = self.transform(data)
                            data_list.append(data)
                else:
                    print(f"Path does not exist: {split_path}")  # Debugging print
        return data_list

    def load_off(self, file_path):
        with open(file_path, 'r') as f:
            if 'OFF' != f.readline().strip():
                raise('Not a valid OFF header')
            n_verts, n_faces, _ = map(int, f.readline().strip().split(' '))
            vertices = [list(map(float, f.readline().strip().split(' '))) for _ in range(n_verts)]
            faces = [list(map(int, f.readline().strip().split(' ')[1:])) for _ in range(n_faces)]
        return vertices, faces

    def process_data(self, vertices, cls):
        x = torch.tensor(vertices, dtype=torch.float)
        y = torch.tensor([self.classes.index(cls)], dtype=torch.long)
        edge_index = knn_graph(x, k=6)
        return Data(x=x, edge_index=edge_index, y=y)

# Specify dataset root and classes
root = 'C:/Users/Owner/Downloads/ModelNet10/ModelNet10'
classes = ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']

# Initialize and load dataset
dataset = ModelNetDataset(root, classes, transform=KNNGraph(k=6))
data_list = dataset.load_data()

# Debugging print
print(f"Total data samples loaded: {len(data_list)}")

# Create data loader
data_loader = DataLoader(data_list, batch_size=1, shuffle=True)

# Print to confirm the data loader is working as expected
for data in data_loader:
    print(data)
    break

Checking path: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0001.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0002.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0003.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0004.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0005.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0006.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0007.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0008.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0009.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/batht

Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0087.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0088.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0089.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0090.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0091.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0092.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0093.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0094.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0095.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bathtub/train/bathtub_0096.off
Processing file: C:/Users/Owner/Downloads/ModelNet

Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0021.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0022.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0023.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0024.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0025.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0026.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0027.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0028.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0029.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0030.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0031.off
Processing file: C:/Users/Owner/Downloads/M

Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0119.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0120.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0121.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0122.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0123.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0124.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0125.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0126.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0127.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0128.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0129.off
Processing file: C:/Users/Owner/Downloads/M

Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0219.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0220.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0221.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0222.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0223.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0224.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0225.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0226.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0227.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0228.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/bed/train/bed_0229.off
Processing file: C:/Users/Owner/Downloads/M

...

Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0171.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0172.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0173.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0174.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0175.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0176.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0177.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0178.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0179.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0180.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet

Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0261.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0262.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0263.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0264.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0265.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0266.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0267.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0268.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0269.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/train/toilet_0270.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet

Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0352.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0353.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0354.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0355.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0356.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0357.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0358.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0359.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0360.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0361.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toil

Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0442.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0443.off
Processing file: C:/Users/Owner/Downloads/ModelNet10/ModelNet10/toilet/test/toilet_0444.off
Total data samples loaded: 4899
DataBatch(x=[1098, 3], edge_index=[2, 6588], y=[1], pos=[1098, 3], batch=[1098], ptr=[2])


C:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [2]:
# Define the Graph VAE
class GraphVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(GraphVAE, self).__init__()
        self.encoder = GCNConv(input_dim, hidden_dim)
        self.mu_layer = GCNConv(hidden_dim, latent_dim)
        self.logvar_layer = GCNConv(hidden_dim, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x, edge_index):
        h = F.relu(self.encoder(x, edge_index))
        mu = self.mu_layer(h, edge_index)
        logvar = self.logvar_layer(h, edge_index)
        z = self.reparameterize(mu, logvar)
        x_reconstructed = self.decoder(z)
        return x_reconstructed, mu, logvar

# Define a pre-trained Graph Neural Network for FID computation
class PretrainedGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PretrainedGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
    
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Function to calculate FID
def calculate_fid(real_embeddings, fake_embeddings):
    mu_real = np.mean(real_embeddings, axis=0)
    sigma_real = np.cov(real_embeddings, rowvar=False)
    mu_fake = np.mean(fake_embeddings, axis=0)
    sigma_fake = np.cov(fake_embeddings, rowvar=False)
    
    ssdiff = np.sum((mu_real - mu_fake)**2.0)
    covmean = sqrtm(sigma_real.dot(sigma_fake))
    
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    fid = ssdiff + np.trace(sigma_real + sigma_fake - 2.0 * covmean)
    return fid

In [ ]:
# Training Loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dim = 3  # Dimension of node features
hidden_dim = 64
latent_dim = 32
num_epochs = 100

vae = GraphVAE(input_dim, hidden_dim, latent_dim).to(device)
pretrained_gnn = PretrainedGNN(input_dim, hidden_dim, 128).to(device)  # Output embedding dimension is 128

# Use DataParallel to leverage multiple GPUs if available
if torch.cuda.device_count() > 1:
    vae = nn.DataParallel(vae)
    pretrained_gnn = nn.DataParallel(pretrained_gnn)

optimizer = torch.optim.Adam(vae.parameters(), lr=0.001)
criterion = nn.MSELoss()  # Reconstruction loss

def train_vae(vae, pretrained_gnn, data_loader, optimizer, criterion, num_epochs):
    vae.train()
    pretrained_gnn.eval()
    
    epoch_losses = []
    node_accuracies = []
    fid_scores = []
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        correct_nodes = 0
        total_nodes = 0
        
        real_embeddings = []
        fake_embeddings = []
        
        with tqdm(total=len(data_loader), desc=f"Epoch {epoch + 1}/{num_epochs}") as pbar:
            for data in data_loader:
                data = data.to(device)
                optimizer.zero_grad()
                
                # Forward pass
                x_reconstructed, mu, logvar = vae(data.x, data.edge_index)
                
                # Reconstruction loss
                loss_reconstruction = criterion(x_reconstructed, data.x)
                
                # KL divergence
                kl_divergence = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
                
                # Total loss
                loss = loss_reconstruction + kl_divergence
                loss.backward()
                optimizer.step()
                
                # Extract embeddings for FID
                with torch.no_grad():
                    real_embedding = pretrained_gnn(data.x, data.edge_index)
                    fake_embedding = pretrained_gnn(x_reconstructed, data.edge_index)
                    real_embeddings.append(real_embedding.cpu().numpy())
                    fake_embeddings.append(fake_embedding.cpu().numpy())
                
                # Metrics
                epoch_loss += loss.item()
                
                # Node-level accuracy (optional, since this is not a classifier)
                # Just for demonstration, we consider reconstruction accuracy
                predicted_labels = torch.argmax(x_reconstructed, dim=1)
                correct_nodes += (predicted_labels == torch.argmax(data.x, dim=1)).sum().item()
                total_nodes += data.x.size(0)
                
                pbar.update(1)
        
        epoch_loss /= len(data_loader)
        node_accuracy = 100. * correct_nodes / total_nodes
        
        epoch_losses.append(epoch_loss)
        node_accuracies.append(node_accuracy)
        
        # Calculate FID
        real_embeddings = np.concatenate(real_embeddings, axis=0)
        fake_embeddings = np.concatenate(fake_embeddings, axis=0)
        fid_score = calculate_fid(real_embeddings, fake_embeddings)
        fid_scores.append(fid_score)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Node Accuracy: {node_accuracy:.2f}%, FID: {fid_score:.4f}")
    
    return epoch_losses, node_accuracies, fid_scores

# Run the training loop
epoch_losses, node_accuracies, fid_scores = train_vae(vae, pretrained_gnn, data_loader, optimizer, criterion, num_epochs)


Epoch 1/100:  37%|████████████████▋                            | 1819/4899 [01:30<01:58, 25.97it/s]

In [ ]:
# Plot the results
plt.figure(figsize=(12, 10))

plt.subplot(3, 1, 1)
plt.plot(epoch_losses, label='Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss per Epoch')
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(node_accuracies, label='Node Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Node Level Accuracy per Epoch')
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(fid_scores, label='FID Score')
plt.xlabel('Epoch')
plt.ylabel('FID')
plt.title('Frechet Inception Distance (FID) per Epoch')
plt.legend()

plt.tight_layout()
plt.show()